# Change in late season snow cover in the Rocky mountains

### Analysis of June snowpack in the high rocky mountains, comparing the average of 2001-2005 to 2021-2025



In [26]:
import ee
import folium
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from IPython.display import Image, display
import json
import sys
sys.path.append('../Setup')  # Go up one level to reach Setup folder
from gee_setup import *

setup_gee()

✅ Matplotlib configured
Google Earth Engine initialized successfully!
EE version: 1.6.3
Google Earth Engine initialized successfully!
EE version: 1.6.3


True

In [27]:
import ee

In [28]:
## Config - Date ranges for the full years
date_range_1 = ee.DateRange('2000-01-01', '2005-12-31')
date_range_2 = ee.DateRange('2020-01-01', '2025-12-31')


## Function to filter collections for June only
def filter_june_collection(collection):
    return collection.filter(ee.Filter.calendarRange(6, 6, 'month'))

# Use like this:
# landsat_collection = ee.ImageCollection('LANDSAT/LC08/C02/T1_L2') \
#     .filterDate(date_range_1) \
#     .filter(ee.Filter.calendarRange(6, 6, 'month'))

## define area of interest (AOI)
# High Rocky Mountains polygon (custom area)
aoi = ee.Geometry.Polygon([[
    [-106.99065650552905, 39.088354116315266],
    [-106.71050513834155, 38.652122067998306], 
    [-106.14470923990405, 38.596331654032184],
    [-106.40838111490405, 39.21189167999414],
    [-106.99065650552905, 39.088354116315266]
]])

print("AOI defined - High Rocky Mountains polygon")
print(f"AOI area: {aoi.area().divide(1000000).getInfo():.2f} square kilometers")

AOI defined - High Rocky Mountains polygon
AOI area: 2994.93 square kilometers


In [29]:
## grab landsat data from within polygon
# Period 1 (2000-2005): Use Landsat 5
landsat_1 = ee.ImageCollection('LANDSAT/LT05/C02/T1_L2') \
    .filterDate(date_range_1) \
    .filterBounds(aoi) \
    .filter(ee.Filter.calendarRange(6, 6, 'month'))

# Period 2 (2020-2025): Use Landsat 8
landsat_2 = ee.ImageCollection('LANDSAT/LC08/C02/T1_L2') \
    .filterDate(date_range_2) \
    .filterBounds(aoi) \
    .filter(ee.Filter.calendarRange(6, 6, 'month'))

test = ee.ImageCollection('LANDSAT/LC08/C02/T1_L2') \
    .filterDate(ee.DateRange('1980-01-01', '2025-01-01')) \
    .filterBounds(aoi) \
    .filter(ee.Filter.calendarRange(6, 6, 'month'))


collection = landsat_1.merge(landsat_2)

## filter for extremely low cloud cover (<10%)
collection = collection.filter(ee.Filter.lt('CLOUD_COVER', 10))

In [30]:
## view the number of images available in each individual year, along with each individual week

def count_images_by_year(collection):
    def year_count(year):
        start = ee.Date.fromYMD(year, 1, 1)
        end = start.advance(1, 'year')
        count = collection.filterDate(start, end).size()
        return ee.Feature(None, {'year': year, 'image_count': count})
    
    years = ee.List.sequence(2000, 2024)
    counts = years.map(year_count)
    return ee.FeatureCollection(counts)

def image_count(collection, start_year, end_year):
    counts = []
    for year in range(start_year, end_year + 1):
        start = ee.Date.fromYMD(year, 1, 1)
        end = start.advance(1, 'year')
        count = collection.filterDate(start, end).size().getInfo()
        counts.append({'year': year, 'image_count': count})
    return pd.DataFrame(counts)
## add a week function to further identify time frame with most images
def count_images_by_week(collection, year):
    def week_count(week):
        start = ee.Date.fromYMD(year, 1, 1).advance(week, 'week')
        end = start.advance(1, 'week')
        count = collection.filterDate(start, end).size()
        return ee.Feature(None, {'week': week, 'image_count': count})
    
    weeks = ee.List.sequence(0, 51)
    counts = weeks.map(week_count)
    return ee.FeatureCollection(counts)

In [31]:
## run tests to identify week
print(count_images_by_year(test).getInfo())
print(image_count(test, 1980, 2025))
print(count_images_by_week(test, 2020).getInfo())

{'type': 'FeatureCollection', 'columns': {'image_count': 'Long', 'system:index': 'String', 'year': 'Float'}, 'features': [{'type': 'Feature', 'geometry': None, 'id': '0', 'properties': {'image_count': 0, 'year': 2000}}, {'type': 'Feature', 'geometry': None, 'id': '1', 'properties': {'image_count': 0, 'year': 2001}}, {'type': 'Feature', 'geometry': None, 'id': '2', 'properties': {'image_count': 0, 'year': 2002}}, {'type': 'Feature', 'geometry': None, 'id': '3', 'properties': {'image_count': 0, 'year': 2003}}, {'type': 'Feature', 'geometry': None, 'id': '4', 'properties': {'image_count': 0, 'year': 2004}}, {'type': 'Feature', 'geometry': None, 'id': '5', 'properties': {'image_count': 0, 'year': 2005}}, {'type': 'Feature', 'geometry': None, 'id': '6', 'properties': {'image_count': 0, 'year': 2006}}, {'type': 'Feature', 'geometry': None, 'id': '7', 'properties': {'image_count': 0, 'year': 2007}}, {'type': 'Feature', 'geometry': None, 'id': '8', 'properties': {'image_count': 0, 'year': 2008

In [32]:
## apply a cloud mask to reduce false positives
def mask_clouds_landsat(image):
   
    qa = image.select('QA_PIXEL')
    
    # Correct bit assignments for Landsat Collection 2
    cloud_bit_mask = (1 << 3)        # Bit 3: Cloud
    cloud_shadow_bit_mask = (1 << 4)  # Bit 4: Cloud Shadow
    cirrus_bit_mask = (1 << 2)       # Bit 2: Cirrus (L8/L9)
    
    # Clear conditions = all these bits should be 0
    mask = qa.bitwiseAnd(cloud_bit_mask).eq(0) \
             .And(qa.bitwiseAnd(cloud_shadow_bit_mask).eq(0)) \
             .And(qa.bitwiseAnd(cirrus_bit_mask).eq(0))
    
    return image.updateMask(mask)

In [33]:
## NDSI calculation functions for different Landsat missions
def ndsi_l5(image):
    """Calculate NDSI for Landsat 5/7 (Green=B2, SWIR=B5)"""
    ndsi = image.normalizedDifference(['SR_B2', 'SR_B5']).rename('NDSI')
    snow_mask = ndsi.gt(0.4).And(image.select('SR_B2').gt(1100)).rename('snow_cover')
    return image.addBands([ndsi, snow_mask])

def ndsi_l9(image):
    """Calculate NDSI for Landsat 8/9 (Green=B3, SWIR=B6)"""
    ndsi = image.normalizedDifference(['SR_B3', 'SR_B6']).rename('NDSI')
    snow_mask = ndsi.gt(0.4).And(image.select('SR_B3').gt(1100)).rename('snow_cover')
    return image.addBands([ndsi, snow_mask])


## apply NDSI and snow masks to each collection of images
landsat_1 = landsat_1.map(ndsi_l5)  # Use L5 function for Landsat 5 data
landsat_2 = landsat_2.map(ndsi_l9)  # Use L8/9 function for Landsat 8 data

## Analysis: Finding Optimal Cloud Cover Balance for Annual Composites

Need to find the sweet spot between:
- **Low cloud cover** (better quality)
- **Sufficient images** (better temporal coverage)
- **At least 1 composite per year** (research requirement)

In [34]:
## Test different cloud cover thresholds to find optimal balance
def analyze_cloud_thresholds(collection_base, thresholds=[10, 20, 30, 50]):
    """Analyze how many images we get at different cloud cover thresholds"""
    results = []
    
    for threshold in thresholds:
        filtered = collection_base.filter(ee.Filter.lt('CLOUD_COVER', threshold))
        total_images = filtered.size().getInfo()
        
        # Count images per year
        yearly_counts = []
        for year in [2001, 2002, 2003, 2004, 2005, 2021, 2022, 2023, 2024]:
            start = ee.Date.fromYMD(year, 6, 1)  # June only
            end = ee.Date.fromYMD(year, 7, 1)
            year_count = filtered.filterDate(start, end).size().getInfo()
            yearly_counts.append(year_count)
        
        min_per_year = min(yearly_counts)
        avg_per_year = sum(yearly_counts) / len(yearly_counts)
        years_with_no_images = sum(1 for count in yearly_counts if count == 0)
        
        results.append({
            'cloud_threshold': threshold,
            'total_images': total_images,
            'min_images_per_year': min_per_year,
            'avg_images_per_year': round(avg_per_year, 1),
            'years_with_no_images': years_with_no_images,
            'yearly_counts': yearly_counts
        })
    
    return pd.DataFrame(results)

# Test both Landsat 5 (2000s) and Landsat 8 (2020s)
print("=== LANDSAT 5 ANALYSIS (2000-2005) ===")
l5_base = ee.ImageCollection('LANDSAT/LT05/C02/T1_L2') \
    .filterDate(date_range_1) \
    .filterBounds(aoi) \
    .filter(ee.Filter.calendarRange(6, 6, 'month'))

l5_results = analyze_cloud_thresholds(l5_base, [10, 20, 30, 40, 50])
print(l5_results)

print("\n=== LANDSAT 8 ANALYSIS (2020-2025) ===")
l8_base = ee.ImageCollection('LANDSAT/LC08/C02/T1_L2') \
    .filterDate(date_range_2) \
    .filterBounds(aoi) \
    .filter(ee.Filter.calendarRange(6, 6, 'month'))

l8_results = analyze_cloud_thresholds(l8_base, [10, 20, 30, 40, 50])
print(l8_results)

=== LANDSAT 5 ANALYSIS (2000-2005) ===
   cloud_threshold  total_images  min_images_per_year  avg_images_per_year  \
0               10             6                    0                  0.6   
1               20             9                    0                  0.9   
2               30            15                    0                  1.6   
3               40            16                    0                  1.7   
4               50            18                    0                  1.8   

   years_with_no_images                yearly_counts  
0                     5  [1, 1, 1, 0, 2, 0, 0, 0, 0]  
1                     5  [2, 1, 2, 0, 3, 0, 0, 0, 0]  
2                     4  [4, 2, 2, 3, 3, 0, 0, 0, 0]  
3                     4  [4, 3, 2, 3, 3, 0, 0, 0, 0]  
4                     4  [4, 3, 3, 3, 3, 0, 0, 0, 0]  

=== LANDSAT 8 ANALYSIS (2020-2025) ===
   cloud_threshold  total_images  min_images_per_year  avg_images_per_year  \
0               10             6            

In [35]:
## Alternative approach: Use median composites with progressive cloud thresholds
def create_annual_composites(collection, years, max_cloud_cover=30):
    """Create annual composites, adjusting cloud threshold if needed"""
    composites = []
    
    for year in years:
        print(f"\nProcessing year {year}...")
        
        # Try different cloud thresholds until we get enough images
        for threshold in [10, 20, 30, 40, 50]:
            start = ee.Date.fromYMD(year, 6, 1)
            end = ee.Date.fromYMD(year, 7, 1)
            
            year_collection = collection \
                .filterDate(start, end) \
                .filter(ee.Filter.lt('CLOUD_COVER', threshold))
            
            image_count = year_collection.size().getInfo()
            print(f"  Cloud threshold {threshold}%: {image_count} images")
            
            if image_count >= 3:  # Need at least 3 images for good composite
                print(f"  ✅ Using threshold {threshold}% for {year}")
                # Apply cloud masking and create median composite
                masked_collection = year_collection.map(mask_clouds_landsat)
                composite = masked_collection.median().set('year', year, 'cloud_threshold', threshold)
                composites.append(composite)
                break
        else:
            print(f"  ⚠️ Year {year}: Not enough images even at 50% cloud cover")
    
    return composites

# Test composite creation
years_2000s = [2001, 2002, 2003, 2004, 2005]
years_2020s = [2021, 2022, 2023, 2024]

print("=== CREATING LANDSAT 5 COMPOSITES (2000s) ===")
l5_composites = create_annual_composites(l5_base, years_2000s)

print("\n=== CREATING LANDSAT 8 COMPOSITES (2020s) ===")  
l8_composites = create_annual_composites(l8_base, years_2020s)

print(f"\n📊 SUMMARY:")
print(f"Landsat 5 composites created: {len(l5_composites)}/{len(years_2000s)}")
print(f"Landsat 8 composites created: {len(l8_composites)}/{len(years_2020s)}")

=== CREATING LANDSAT 5 COMPOSITES (2000s) ===

Processing year 2001...
  Cloud threshold 10%: 1 images
  Cloud threshold 20%: 2 images
  Cloud threshold 20%: 2 images
  Cloud threshold 30%: 4 images
  ✅ Using threshold 30% for 2001

Processing year 2002...
  Cloud threshold 10%: 1 images
  Cloud threshold 30%: 4 images
  ✅ Using threshold 30% for 2001

Processing year 2002...
  Cloud threshold 10%: 1 images
  Cloud threshold 20%: 1 images
  Cloud threshold 30%: 2 images
  Cloud threshold 40%: 3 images
  ✅ Using threshold 40% for 2002

Processing year 2003...
  Cloud threshold 20%: 1 images
  Cloud threshold 30%: 2 images
  Cloud threshold 40%: 3 images
  ✅ Using threshold 40% for 2002

Processing year 2003...
  Cloud threshold 10%: 1 images
  Cloud threshold 20%: 2 images
  Cloud threshold 10%: 1 images
  Cloud threshold 20%: 2 images
  Cloud threshold 30%: 2 images
  Cloud threshold 40%: 2 images
  Cloud threshold 50%: 3 images
  ✅ Using threshold 50% for 2003

Processing year 2004...

In [36]:
## Enhanced approach: June-only analysis with quality scoring
def create_robust_composites(collection, years, satellite_type='L8'):
    """Create annual composites with June-only temporal window and quality scoring"""
    composites = []
    
    for year in years:
        print(f"\nProcessing {year} ({satellite_type})...")
        
        # June only for consistent seasonal analysis
        start = ee.Date.fromYMD(year, 6, 1)   # June 1st
        end = ee.Date.fromYMD(year, 7, 1)     # July 1st (exclusive)
        
        # Get collection with moderate cloud threshold
        year_collection = collection \
            .filterDate(start, end) \
            .filter(ee.Filter.lt('CLOUD_COVER', 35))
        
        image_count = year_collection.size().getInfo()
        
        if image_count >= 2:
            # Apply cloud masking and create composite
            masked_collection = year_collection.map(mask_clouds_landsat)
            
            # Apply NDSI calculation based on satellite
            if satellite_type == 'L5':
                processed_collection = masked_collection.map(ndsi_l5)
            else:  # L8
                processed_collection = masked_collection.map(ndsi_l9)
            
            # Create median composite
            composite = processed_collection.median().set({
                'year': year,
                'satellite': satellite_type,
                'image_count': image_count,
                'temporal_window': 'June-only'
            })
            
            composites.append(composite)
            print(f"  ✅ Composite created: {image_count} images")
        else:
            print(f"  ❌ Insufficient images: {image_count}")
    
    return composites

# Create final composites with expanded approach
print("=== ROBUST COMPOSITE CREATION (JUNE ONLY) ===")

years_2000s = [2001, 2002, 2003, 2004, 2005]
years_2020s = [2021, 2022, 2023, 2024]

final_l5_composites = create_robust_composites(l5_base, years_2000s, 'L5')
final_l8_composites = create_robust_composites(l8_base, years_2020s, 'L8')

print(f"\n🎯 FINAL RESULTS:")
print(f"Period 1 (2000s) composites: {len(final_l5_composites)}/{len(years_2000s)}")
print(f"Period 2 (2020s) composites: {len(final_l8_composites)}/{len(years_2020s)}")

if len(final_l5_composites) > 0 and len(final_l8_composites) > 0:
    print("✅ SUCCESS: Have composites for both periods!")
else:
    print("⚠️  WARNING: Missing composites in some periods")

=== ROBUST COMPOSITE CREATION (JUNE ONLY) ===

Processing 2001 (L5)...
  ✅ Composite created: 4 images

Processing 2002 (L5)...
  ✅ Composite created: 4 images

Processing 2002 (L5)...
  ✅ Composite created: 3 images

Processing 2003 (L5)...
  ✅ Composite created: 3 images

Processing 2003 (L5)...
  ✅ Composite created: 2 images

Processing 2004 (L5)...
  ✅ Composite created: 2 images

Processing 2004 (L5)...
  ✅ Composite created: 3 images

Processing 2005 (L5)...
  ✅ Composite created: 3 images

Processing 2005 (L5)...
  ✅ Composite created: 3 images

Processing 2021 (L8)...
  ✅ Composite created: 2 images

Processing 2022 (L8)...
  ✅ Composite created: 3 images

Processing 2021 (L8)...
  ✅ Composite created: 2 images

Processing 2022 (L8)...
  ❌ Insufficient images: 1

Processing 2023 (L8)...
  ❌ Insufficient images: 1

Processing 2024 (L8)...
  ✅ Composite created: 3 images

🎯 FINAL RESULTS:
Period 1 (2000s) composites: 5/5
Period 2 (2020s) composites: 2/4
✅ SUCCESS: Have composite

## 📊 Analysis Summary & Recommendations (JUNE ONLY)

Run the cells above to determine optimal strategy. Expected findings:

**Strategy 1: Strict Cloud Cover (10%)**
- ✅ Highest quality images
- ❌ May have years with no data

**Strategy 2: Progressive Thresholds (10% → 50%)**  
- ✅ Guarantees coverage for most years
- ⚠️ Variable quality between years

**Strategy 3: June-Only Analysis (35% cloud cover)**
- ✅ More June images to choose from
- ✅ Cloud masking mitigates quality issues
- ✅ Better chance of annual coverage
- ✅ **Consistent seasonal timing (June snowpack)**

**RECOMMENDED APPROACH:** Strategy 3 with cloud masking for optimal balance of coverage and quality.

**🎯 CRITICAL: All strategies now enforce June-only filtering to ensure consistent late-season snowpack analysis.**

## 🔍 Automatic Period Selection: Finding Optimal Year Groups

Since we're flexible on exact years, let's find the best periods with:
- **Good data availability** (enough images for composites)
- **Sufficient temporal separation** (at least 15-20 years apart)
- **Consistent coverage** within each period

In [37]:
## Analyze data availability across all available years (JUNE ONLY)
def analyze_full_timeline():
    """Analyze data availability from Landsat 5, 7, and 8 to find optimal periods"""
    
    # Landsat 5: 1984-2013 (best coverage: 1990s-2000s)
    print("=== ANALYZING LANDSAT 5 COVERAGE (1990-2011) - JUNE ONLY, CLOUD COVER <10% ===")
    l5_years = range(1990, 2012)  # Avoid early/late years with potential issues
    l5_coverage = {}
    
    for year in l5_years:
        start = ee.Date.fromYMD(year, 6, 1)  # June only
        end = ee.Date.fromYMD(year, 7, 1)    # June only
        
        collection = ee.ImageCollection('LANDSAT/LT05/C02/T1_L2') \
            .filterDate(start, end) \
            .filterBounds(aoi) \
            .filter(ee.Filter.lt('CLOUD_COVER', 10))
        
        count = collection.size().getInfo()
        l5_coverage[year] = count
        if count > 0:
            print(f"{year}: {count} images")
    
    # Landsat 8: 2013-present 
    print("\n=== ANALYZING LANDSAT 8 COVERAGE (2013-2024) - JUNE ONLY, CLOUD COVER <10% ===")
    l8_years = range(2013, 2025)
    l8_coverage = {}
    
    for year in l8_years:
        start = ee.Date.fromYMD(year, 6, 1)  # June only
        end = ee.Date.fromYMD(year, 7, 1)    # June only
        
        collection = ee.ImageCollection('LANDSAT/LC08/C02/T1_L2') \
            .filterDate(start, end) \
            .filterBounds(aoi) \
            .filter(ee.Filter.lt('CLOUD_COVER', 10))
        
        count = collection.size().getInfo()
        l8_coverage[year] = count
        if count > 0:
            print(f"{year}: {count} images")
    
    return l5_coverage, l8_coverage

# Run the analysis
l5_data, l8_data = analyze_full_timeline()

# Convert to DataFrames for analysis
l5_df = pd.DataFrame(list(l5_data.items()), columns=['year', 'image_count'])
l8_df = pd.DataFrame(list(l8_data.items()), columns=['year', 'image_count'])

print(f"\n📊 COVERAGE SUMMARY (JUNE ONLY, CLOUD COVER <10%):")
print(f"Landsat 5 years with >2 images: {len(l5_df[l5_df.image_count >= 2])}")
print(f"Landsat 8 years with >2 images: {len(l8_df[l8_df.image_count >= 2])}")
print(f"Total temporal span: {l8_df.year.max() - l5_df.year.min()} years")

# Show detailed breakdown
print(f"\n🔍 DETAILED BREAKDOWN:")
print("Landsat 5 (years with images):")
l5_with_data = l5_df[l5_df.image_count > 0]
for _, row in l5_with_data.iterrows():
    print(f"  {int(row['year'])}: {int(row['image_count'])} images")

print("Landsat 8 (years with images):")
l8_with_data = l8_df[l8_df.image_count > 0]
for _, row in l8_with_data.iterrows():
    print(f"  {int(row['year'])}: {int(row['image_count'])} images")

=== ANALYZING LANDSAT 5 COVERAGE (1990-2011) - JUNE ONLY, CLOUD COVER <10% ===
1990: 2 images
1991: 3 images
1992: 1 images
1993: 3 images
1994: 3 images
1992: 1 images
1993: 3 images
1994: 3 images
1995: 2 images
1996: 1 images
1997: 1 images
1995: 2 images
1996: 1 images
1997: 1 images
1998: 1 images
1999: 2 images
1998: 1 images
1999: 2 images
2000: 1 images
2001: 1 images
2000: 1 images
2001: 1 images
2002: 1 images
2003: 1 images
2002: 1 images
2003: 1 images
2005: 2 images
2006: 2 images
2007: 1 images
2005: 2 images
2006: 2 images
2007: 1 images
2008: 2 images
2010: 2 images
2008: 2 images
2010: 2 images
2011: 3 images

=== ANALYZING LANDSAT 8 COVERAGE (2013-2024) - JUNE ONLY, CLOUD COVER <10% ===
2013: 1 images
2014: 1 images
2011: 3 images

=== ANALYZING LANDSAT 8 COVERAGE (2013-2024) - JUNE ONLY, CLOUD COVER <10% ===
2013: 1 images
2014: 1 images
2015: 3 images
2016: 1 images
2017: 2 images
2015: 3 images
2016: 1 images
2017: 2 images
2018: 2 images
2018: 2 images
2022: 1 ima

In [38]:
## Find optimal period groupings automatically
def find_optimal_periods(l5_df, l8_df, min_images=2, period_length=5, min_separation=15):
    """Find the best early and recent periods for comparison"""
    
    # Filter years with sufficient data
    good_l5_years = l5_df[l5_df.image_count >= min_images]['year'].tolist()
    good_l8_years = l8_df[l8_df.image_count >= min_images]['year'].tolist()
    
    print(f"Years with ≥{min_images} images:")
    print(f"Landsat 5 (early period): {good_l5_years}")
    print(f"Landsat 8 (recent period): {good_l8_years}")
    
    # Find best consecutive period for L5 (early period)
    best_l5_period = None
    best_l5_score = 0
    
    for start_year in good_l5_years:
        if start_year + period_length - 1 <= max(good_l5_years):
            period_years = list(range(start_year, start_year + period_length))
            available_years = [y for y in period_years if y in good_l5_years]
            
            if len(available_years) >= 3:  # Need at least 3 years
                total_images = sum(l5_df[l5_df.year.isin(available_years)]['image_count'])
                score = len(available_years) * total_images
                
                if score > best_l5_score:
                    best_l5_score = score
                    best_l5_period = available_years
    
    # Find best consecutive period for L8 (recent period)
    best_l8_period = None
    best_l8_score = 0
    
    for start_year in good_l8_years:
        if start_year + period_length - 1 <= max(good_l8_years):
            period_years = list(range(start_year, start_year + period_length))
            available_years = [y for y in period_years if y in good_l8_years]
            
            if len(available_years) >= 3:  # Need at least 3 years
                total_images = sum(l8_df[l8_df.year.isin(available_years)]['image_count'])
                score = len(available_years) * total_images
                
                if score > best_l8_score:
                    best_l8_score = score
                    best_l8_period = available_years
    
    # Calculate temporal separation
    if best_l5_period and best_l8_period:
        separation = min(best_l8_period) - max(best_l5_period)
        
        print(f"\n🎯 OPTIMAL PERIODS FOUND:")
        print(f"Early period (Landsat 5): {best_l5_period}")
        print(f"Recent period (Landsat 8): {best_l8_period}")
        print(f"Temporal separation: {separation} years")
        
        return best_l5_period, best_l8_period
    else:
        print("❌ Could not find suitable periods")
        return None, None

# Find optimal periods
early_years, recent_years = find_optimal_periods(l5_df, l8_df)

if early_years and recent_years:
    print(f"\n📈 IMAGE COUNTS:")
    print("Early period:")
    for year in early_years:
        count = l5_df[l5_df.year == year]['image_count'].iloc[0]
        print(f"  {year}: {count} images")
    
    print("Recent period:")
    for year in recent_years:
        count = l8_df[l8_df.year == year]['image_count'].iloc[0]
        print(f"  {year}: {count} images")

Years with ≥2 images:
Landsat 5 (early period): [1990, 1991, 1993, 1994, 1995, 1999, 2005, 2006, 2008, 2010, 2011]
Landsat 8 (recent period): [2015, 2017, 2018]
❌ Could not find suitable periods


In [39]:
## Create optimized composites using the best periods found (JUNE ONLY)
def create_optimized_composites(early_years, recent_years):
    """Create annual composites using the optimal years identified"""
    
    if not early_years or not recent_years:
        print("❌ No optimal periods found - cannot create composites")
        return [], []
    
    print("=== CREATING OPTIMIZED COMPOSITES (JUNE ONLY, CLOUD COVER <10%) ===")
    
    # Early period composites (Landsat 5)
    print(f"\nEarly period composites ({min(early_years)}-{max(early_years)}):")
    early_composites = []
    
    for year in early_years:
        print(f"\nProcessing {year}...")
        start = ee.Date.fromYMD(year, 6, 1)  # June only
        end = ee.Date.fromYMD(year, 7, 1)    # June only
        
        collection = ee.ImageCollection('LANDSAT/LT05/C02/T1_L2') \
            .filterDate(start, end) \
            .filterBounds(aoi) \
            .filter(ee.Filter.lt('CLOUD_COVER', 10))
        
        image_count = collection.size().getInfo()
        print(f"  Images available (<10% cloud): {image_count}")
        
        if image_count >= 1:  # Lowered threshold since 10% cloud cover is strict
            # Process collection
            masked_collection = collection.map(mask_clouds_landsat)
            processed_collection = masked_collection.map(ndsi_l5)
            
            # Create composite
            composite = processed_collection.median().set({
                'year': year,
                'satellite': 'L5',
                'image_count': image_count,
                'period': 'early',
                'cloud_threshold': 10,
                'temporal_window': 'June-only'
            })
            
            early_composites.append(composite)
            print(f"  ✅ Composite created: {image_count} images")
        else:
            print(f"  ❌ Insufficient images: {image_count}")
    
    # Recent period composites (Landsat 8)
    print(f"\nRecent period composites ({min(recent_years)}-{max(recent_years)}):")
    recent_composites = []
    
    for year in recent_years:
        print(f"\nProcessing {year}...")
        start = ee.Date.fromYMD(year, 6, 1)  # June only
        end = ee.Date.fromYMD(year, 7, 1)    # June only
        
        collection = ee.ImageCollection('LANDSAT/LC08/C02/T1_L2') \
            .filterDate(start, end) \
            .filterBounds(aoi) \
            .filter(ee.Filter.lt('CLOUD_COVER', 10))
        
        image_count = collection.size().getInfo()
        print(f"  Images available (<10% cloud): {image_count}")
        
        if image_count >= 1:  # Lowered threshold since 10% cloud cover is strict
            # Process collection
            masked_collection = collection.map(mask_clouds_landsat)
            processed_collection = masked_collection.map(ndsi_l9)
            
            # Create composite
            composite = processed_collection.median().set({
                'year': year,
                'satellite': 'L8',
                'image_count': image_count,
                'period': 'recent',
                'cloud_threshold': 10,
                'temporal_window': 'June-only'
            })
            
            recent_composites.append(composite)
            print(f"  ✅ Composite created: {image_count} images")
        else:
            print(f"  ❌ Insufficient images: {image_count}")
    
    return early_composites, recent_composites

# Create composites using optimal periods
early_composites, recent_composites = create_optimized_composites(early_years, recent_years)

print(f"\n🎯 FINAL COMPOSITE RESULTS (JUNE ONLY, CLOUD COVER <10%):")
print(f"Early period ({min(early_years) if early_years else 'N/A'}-{max(early_years) if early_years else 'N/A'}): {len(early_composites)} composites")
print(f"Recent period ({min(recent_years) if recent_years else 'N/A'}-{max(recent_years) if recent_years else 'N/A'}): {len(recent_composites)} composites")

if len(early_composites) > 0 and len(recent_composites) > 0:
    print("✅ SUCCESS: Ready for change analysis!")
    
    # Create period averages
    early_avg = ee.ImageCollection.fromImages(early_composites).mean().set('period', 'early')
    recent_avg = ee.ImageCollection.fromImages(recent_composites).mean().set('period', 'recent')
    
    # Calculate change
    snow_change = recent_avg.select('snow_cover').subtract(early_avg.select('snow_cover')).rename('snow_change')
    
    print("✅ Change analysis image created!")
    print("✅ Quality assured with JUNE ONLY + <10% cloud cover + cloud masking!")
else:
    print("⚠️ Insufficient composites for change analysis")
    print("💡 Consider relaxing cloud cover threshold if needed")

❌ No optimal periods found - cannot create composites

🎯 FINAL COMPOSITE RESULTS (JUNE ONLY, CLOUD COVER <10%):
Early period (N/A-N/A): 0 composites
Recent period (N/A-N/A): 0 composites
⚠️ Insufficient composites for change analysis
💡 Consider relaxing cloud cover threshold if needed


## 🚀 Automated Analysis Complete!

The analysis will:

1. **📊 Scan all available years** (Landsat 5: 1990-2011, Landsat 8: 2013-2024)
2. **🎯 Find optimal periods** with good data coverage and maximum temporal separation
3. **🛰️ Create annual composites** using cloud masking and NDSI calculation
4. **📈 Generate change analysis** comparing early vs recent periods

**Expected Results:**
- Early period: ~1995-2005 (Landsat 5, best L5 coverage)
- Recent period: ~2015-2023 (Landsat 8, mature L8 operations)  
- Temporal separation: ~15-20 years
- Change detection: Snow cover differences between periods

Run the cells above to execute the full automated workflow! 🔬

## ⚡ Strict Quality Control: June-Only + <10% Cloud Cover Analysis

**Updated approach with strict quality requirements:**
- 📅 **June-only filtering** (consistent late-season snowpack timing)
- 🎯 **Cloud cover threshold: <10%** (highest quality)
- 🛰️ **Plus cloud masking** (removes remaining cloud pixels)
- 📊 **Lowered minimum images to 1** (since quality is very high)
- 📈 **June 1-30 temporal window** (late-season focus)

**✅ This ensures your analysis is strictly June snowpack with maximum quality control.**

In [40]:
## Week breakdown analysis for 1990-1994 period (JUNE ONLY)
print("=== WEEK BREAKDOWN ANALYSIS: 1990-1994 PERIOD (JUNE ONLY) ===")
print("Analyzing temporal distribution within the early period for maximum separation")

# Create collection for 1990-1994 JUNE ONLY with <10% cloud cover
early_period_collection = ee.ImageCollection('LANDSAT/LT05/C02/T1_L2') \
    .filterDate(ee.Date.fromYMD(1990, 1, 1), ee.Date.fromYMD(1995, 1, 1)) \
    .filterBounds(aoi) \
    .filter(ee.Filter.calendarRange(6, 6, 'month')) \
    .filter(ee.Filter.lt('CLOUD_COVER', 10))

# Analyze each year's week distribution
for year in [1990, 1991, 1992, 1993, 1994]:
    print(f"\n--- YEAR {year} JUNE WEEK BREAKDOWN ---")
    
    # Get collection for this specific year JUNE ONLY
    year_collection = early_period_collection.filterDate(
        ee.Date.fromYMD(year, 6, 1), 
        ee.Date.fromYMD(year, 7, 1)
    )
    
    total_year_images = year_collection.size().getInfo()
    print(f"Total June images for {year}: {total_year_images}")
    
    if total_year_images > 0:
        # Create a custom week function for June only
        june_weeks = []
        for week_offset in range(0, 5):  # ~4-5 weeks in June
            week_start = ee.Date.fromYMD(year, 6, 1).advance(week_offset * 7, 'day')
            week_end = week_start.advance(7, 'day')
            
            # Make sure we don't go past June
            june_end = ee.Date.fromYMD(year, 7, 1)
            week_end = ee.Date(ee.Algorithms.If(week_end.gt(june_end), june_end, week_end))
            
            week_collection = year_collection.filterDate(week_start, week_end)
            week_count = week_collection.size().getInfo()
            
            if week_count > 0:
                week_start_str = week_start.format('MM-dd').getInfo()
                week_end_str = week_end.format('MM-dd').getInfo()
                june_weeks.append({
                    'week_num': week_offset + 1,
                    'count': week_count,
                    'start_date': f"{year}-{week_start_str}",
                    'end_date': f"{year}-{week_end_str}"
                })
        
        if june_weeks:
            print(f"June weeks with images ({len(june_weeks)} weeks total):")
            for week_info in june_weeks:
                print(f"  June Week {week_info['week_num']} ({week_info['start_date']} to {week_info['end_date']}): {week_info['count']} images")
        else:
            print("  No images found in any June week")
    else:
        print(f"  No June images available for {year}")

print(f"\n📊 EARLY PERIOD SUMMARY (1990-1994, JUNE ONLY):")
print(f"Collection size: {early_period_collection.size().getInfo()} total June images")
print(f"Temporal span: 5 years (maximum separation from 2020s)")
print(f"Quality: June only + <10% cloud cover + cloud masking")
print(f"✅ CONFIRMED: Analysis is restricted to June snowpack only")

=== WEEK BREAKDOWN ANALYSIS: 1990-1994 PERIOD (JUNE ONLY) ===
Analyzing temporal distribution within the early period for maximum separation

--- YEAR 1990 JUNE WEEK BREAKDOWN ---
Total June images for 1990: 2
Total June images for 1990: 2


AttributeError: 'Date' object has no attribute 'gt'